In [2]:
# Read env variables from .env file

from dotenv import load_dotenv

load_dotenv(override=True)

True

In [3]:
# from langchain.document_loaders.csv_loader import CSVLoader
import logging
from langchain_openai import AzureChatOpenAI
from pydantic import BaseModel, Field
from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

llm = AzureChatOpenAI(model="gpt-4o-mini", temperature=0, api_version="2025-01-01-preview")

class ImageExtractionInput(BaseModel):
    image_path: str = Field(description="Ruta o URL de la imagen del recibo energético del usuario")
    
class InstallationEstimateInput(BaseModel):
    location: str = Field(description="Ubicación donde se instalarán los paneles (ciudad/región)")
    roof_area: float = Field(description="Área disponible en el techo en metros cuadrados")
    monthly_consumption: float = Field(description="Consumo mensual promedio en kWh")
    budget: float = Field(description="Presupuesto disponible para la instalación en pesos")

@tool("extract_energy_consumption", args_schema=ImageExtractionInput)
def extract_energy_consumption(image_path: str) -> str:
    """
    Extrae datos de consumo energético de una imagen de recibo.
    
    Parámetros:
    - image_path: Ruta o URL de la imagen del recibo energético
    
    Retorna:
    - Un objeto con los datos extraídos del consumo energético incluyendo kWh mensuales y dirección
    """
    # Implementation placeholder
    return "He procesado la imagen y extraído: consumo mensual de 350 kWh, dirección: Av. Principal 123, tarifa: $0.18/kWh"

@tool("estimate_installation_cost", args_schema=InstallationEstimateInput)
def estimate_installation_cost(location: str, roof_area: float, monthly_consumption: float, budget: float) -> str:
    """
    Calcula el costo estimado y ROI de una instalación de paneles solares.
    
    Parámetros:
    - location: Ubicación donde se instalarán los paneles
    - roof_area: Área disponible en metros cuadrados
    - monthly_consumption: Consumo mensual promedio en kWh
    - budget: Presupuesto disponible para la instalación
    
    Retorna:
    - Detalles sobre la instalación recomendada, costo estimado, ROI y tiempo de amortización
    """
    # Implementation placeholder
    return f"Para una instalación en {location} con {roof_area}m² y consumo de {monthly_consumption}kWh:\n- Costo estimado: ${budget * 0.8}\n- ROI: 15%\n- Amortización: 6 años\n- Capacidad: 4.5kW (12 paneles)"

tools = [extract_energy_consumption, estimate_installation_cost]

# # # Define the prompt template for the agent
prefix = """
# Consultor Especializado en Energía Solar

Eres un consultor especializado en instalaciones de paneles solares. Tu objetivo es brindar al usuario información precisa sobre:
1. Análisis de su consumo energético actual
2. Recomendaciones personalizadas para instalación de paneles solares
3. Estimación de costos, retorno de inversión (ROI) y amortización

## Flujo de conversación:
1. Saluda y explica brevemente cómo puedes ayudar
2. Solicita la imagen del recibo de energía del usuario
3. Utiliza la herramienta "extract_energy_consumption" para analizar la imagen
4. Pregunta información adicional necesaria:
   - Ubicación específica (ciudad/región)
   - Área disponible en el techo (metros cuadrados)
   - Presupuesto aproximado
5. Utiliza la herramienta "estimate_installation_cost" con todos los datos recopilados
6. Presenta los resultados de manera clara y responde cualquier duda adicional

## Notas importantes:
- Mantén un tono profesional pero amigable
- Si faltan datos, solicítalos amablemente al usuario
- Explica los términos técnicos de manera accesible
- Proporciona siempre recomendaciones basadas en los datos reales del usuario
- Si no puedes determinar con precisión algún dato, menciona los rangos posibles
"""

memory = MemorySaver()

agent = create_react_agent(
    llm,
    tools=tools,
    checkpointer=memory,
    prompt=prefix,
)

In [4]:
question = "Hola, estoy interesado en un proyecto solar. ¿Podrías ayudarme?"
thread_id = "1234567890"

agent.invoke({"messages": [{"role": "user", "content": question}]}, config={"configurable": {"thread_id": thread_id}}, stream_mode = "values")["messages"][-1].content

'¡Hola! Claro que sí, estaré encantado de ayudarte con tu proyecto solar. Puedo ofrecerte un análisis de tu consumo energético actual, recomendaciones personalizadas para la instalación de paneles solares, y estimaciones de costos, retorno de inversión (ROI) y amortización.\n\nPara comenzar, necesitaría que me envíes una imagen de tu recibo de energía. Esto me permitirá analizar tu consumo energético mensual.'